In [1]:
%run ./load_nnfs_module.py

In [2]:
import numpy as np

from nnfs.datasets import MNISTDigits
from nnfs.activations import ReLU, Softmax
from nnfs.losses import CategoricalCrossentropy
from nnfs.optimizers import Momentum
from nnfs.initializers import HeNormal, GlorotNormal, Zeros
from nnfs.metrics import CategoricalAccuracy
from nnfs.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from nnfs.models import FeedForwardNetwork
from nnfs.utils.preprocessing import OneHotEncoder, normalize

In [3]:
dataset = MNISTDigits()
dataset.download_data()
X_train, y_train, X_test, y_test = dataset.load_data()

Data has already been downloaded.


In [4]:
X_train = X_train.reshape((-1, 1, 28, 28))
X_test = X_test.reshape((-1, 1, 28, 28))
X_train, X_test = normalize(X_train, X_test)

In [5]:
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train)
y_test = ohe.transform(y_test)

In [6]:
model = FeedForwardNetwork()
model.add(Convolution2D(num_filters=16,
                        filter_size=(5, 5), 
                        stride=1,
                        padding=0, 
                        activation=ReLU(), 
                        weights_initializer=HeNormal(), 
                        bias_initializer=Zeros(),
                        input_shape=(1, 28, 28)))
model.add(MaxPooling2D(pool_size=(2, 2),
                       stride=2))
model.add(Convolution2D(num_filters=32,
                        filter_size=(3, 3), 
                        stride=1,
                        padding=0, 
                        activation=ReLU(), 
                        weights_initializer=HeNormal(), 
                        bias_initializer=Zeros()))
model.add(MaxPooling2D(pool_size=(2, 2),
                       stride=2))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(units=64,
                activation=ReLU(),
                weights_initializer=HeNormal(),
                bias_initializer=Zeros()))
model.add(Dense(units=10,
                activation=Softmax(),
                weights_initializer=GlorotNormal(),
                bias_initializer=Zeros()))
model.compile(optimizer=Momentum(learning_rate=0.001, nesterov=True, clip_value=0.05),
              loss=CategoricalCrossentropy(),
              metrics=[CategoricalAccuracy()])

In [7]:
model.summary()

+---------------+--------------------+--------------------+-----------+
| Layer         | Input Shape        | Output Shape       | # Params  |
+---------------+--------------------+--------------------+-----------+
| Convolution2D | (None, 1, 28, 28)  | (None, 16, 24, 24) | 6,400     |
| MaxPooling2D  | (None, 16, 24, 24) | (None, 16, 12, 12) | 0         |
| Convolution2D | (None, 16, 12, 12) | (None, 32, 10, 10) | 147,456   |
| MaxPooling2D  | (None, 32, 10, 10) | (None, 32, 5, 5)   | 0         |
| Flatten       | (None, 32, 5, 5)   | (None, 800)        | 0         |
| Dropout       | (None, 800)        | (None, 800)        | 0         |
| Dense         | (None, 800)        | (None, 64)         | 3,276,800 |
| Dense         | (None, 64)         | (None, 10)         | 6,400     |
+---------------+--------------------+--------------------+-----------+
Total params: 3,437,056


In [8]:
model.fit(X_train, y_train, batch_size=32, epochs=3, verbosity=2)
loss, metrics = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss :", loss)
print("Test Metrics :", metrics)

Epoch 1 | Batch 1875/1875 |██████████| ETA: 00:00 | 7.63batch/s, CategoricalCrossentropy=0.121, CategoricalAccuracy=0.974
Epoch 2 | Batch 1875/1875 |██████████| ETA: 00:00 | 7.77batch/s, CategoricalCrossentropy=0.103, CategoricalAccuracy=0.976
Epoch 3 | Batch 1875/1875 |██████████| ETA: 00:00 | 7.70batch/s, CategoricalCrossentropy=0.099, CategoricalAccuracy=0.982


Test Loss : 0.061833575672296366
Test Metrics : {'CategoricalAccuracy': 0.9837}
